## Lesson 3 Exercise 1: Three Queries and Three Tables for Analytics

### Walk through the basics of creating a table in Apache Cassandra, inserting rows of data, and doing a simple CQL query to validate the information. You will practice Denormalization, and the concept of 1 table per query, which is an encouraged practice with Apache Cassandra. 

In [30]:
# Import Cassandra driver to perform operations on Cassandra DB
import cassandra

### Create a connection to the Cluster (equivalent to DB server in SQL)

In [31]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

### Create a keyspace we have to work with (equivalent to a Database in SQL)

In [7]:
try:
    session.execute("""
                    CREATE KEYSPACE IF NOT EXISTS udacity
                    WITH REPLICATION =
                    { 'class': 'SimpleStrategy', 'replication_factor': 1 } """)
except Exception as e:
    print(e)

### Connect to Keyspace. Comapre this with how we create a new session in Postgres

In [32]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### We would like to start creating a New set of OLAP tables to store the Music Library of albums

#### The following are the queries which the Business would like to use to analyze data.

#### 1. Give every album in the music library that was released in the given year
`SELECT * FROM music_library WHERE year = 1970`

#### 2. Give every album in the music library that was created by a given artist
`SELECT * FROM artist_library WHERE artist_name = "The Beatles"`

#### 3. Give all the information about a music library for a given album
`SELECT * FROM album_library WHERE album_name = "Close To You"`

### Because we have three different queries, we need three different tables to handle the queries.

![music_library](images/image1.png)


![artist_library](images/image2.png)


![album_library](images/image3.png)

#### CREATE THREE TABLES IN KEYSPACE: udacity

In [20]:
try:
    query1 = """ CREATE TABLE IF NOT EXISTS music_library
                (year int, artist_name varchar, album_name text, PRIMARY KEY(year, artist_name)) """
    
    query2 = """ CREATE TABLE IF NOT EXISTS artist_library
                (artist_name varchar, year int, album_name text, PRIMARY KEY(artist_name, year)) """
    
    query3 = """ CREATE TABLE IF NOT EXISTS album_library 
                (album_name text, artist_name varchar, year int, PRIMARY KEY(album_name, artist_name)) """
    
    # CREATE TABLE MUSIC LIBRARY 
    session.execute(query1)
    
    # CREATE TABLE ARTIST LIBRARY
    session.execute(query2)
    
    # CREATE TABLE ALBUM LIBRARY
    session.execute(query3)
    
except Exception as e:
    print(e)
    

### INSERT DATA INTO THREE TABLES IN KEYSPACE: udacity

In [23]:
query = "INSERT INTO music_library (year, artist_name, album_name)"
query = query + " VALUES (%s, %s, %s)"

query1 = "INSERT INTO artist_library (artist_name, year, album_name)"
query1 = query1 + " VALUES (%s, %s, %s)"

query2 = "INSERT INTO album_library (album_name, artist_name, year)"
query2 = query2 + " VALUES (%s, %s, %s)"

In [15]:
try:
    session.execute(query, (1970, "The Beatles", "Let it Be"))
except Exception as e:
    print(e)
    
try:
    session.execute(query, (1965, "The Beatles", "Rubber Soul"))
except Exception as e:
    print(e)
    
try:
    session.execute(query, (1965, "The Who", "My Generation"))
except Exception as e:
    print(e)

try:
    session.execute(query, (1966, "The Monkees", "The Monkees"))
except Exception as e:
    print(e)

try:
    session.execute(query, (1970, "The Carpenters", "Close To You"))
except Exception as e:
    print(e)

In [24]:
try:
    session.execute(query1, ("The Beatles", 1970, "Let it Be"))
except Exception as e:
    print(e)
    
try:
    session.execute(query1, ("The Beatles", 1965, "Rubber Soul"))
except Exception as e:
    print(e)
    
try:
    session.execute(query1, ("The Who", 1965, "My Generation"))
except Exception as e:
    print(e)

try:
    session.execute(query1, ("The Monkees", 1966, "The Monkees"))
except Exception as e:
    print(e)

try:
    session.execute(query1, ("The Carpenters", 1970, "Close To You"))
except Exception as e:
    print(e)

In [ ]:
try:
    session.execute(query2, ("Let it Be", "The Beatles", 1970))
except Exception as e:
    print(e)
    
try:
    session.execute(query2, ("Rubber Soul", "The Beatles", 1965))
except Exception as e:
    print(e)
    
try:
    session.execute(query2, ("My Generation", "The Who", 1965))
except Exception as e:
    print(e)

try:
    session.execute(query2, ("The Monkees", "The Monkees", 1966))
except Exception as e:
    print(e)

try:
    session.execute(query2, ("Close To You", "The Carpenters", 1970))
except Exception as e:
    print(e)

### Validate the Data Model using the SELECT statements

#### VALIDATE MUSIC LIBRARY

In [25]:
query = "select * from music_library where year = 1970"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.year, row.artist_name, row.album_name)

1970 The Beatles Let it Be
1970 The Carpenters Close To You


#### VALIDATE ARTIST_LIBRARY

In [26]:
query = "select * from artist_library where artist_name = 'The Beatles'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist_name, row.year, row.album_name)

The Beatles 1965 Rubber Soul
The Beatles 1970 Let it Be


#### VALIDATE ALBUM LIBRARY

In [28]:
query = "select * from album_library where album_name = 'Close To You'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.album_name, row.artist_name, row.year)

Close To You The Carpenters 1970


#### VALIDATION FROM Shell Script using CQLSH

![VALIDATION_THROUGH_CQLSH](Validation_shellscript.png)

### DROP THE TABLES

In [33]:
# Drop Music Library table
try:
    query = "DROP TABLE music_library"
    session.execute(query)
except Exception as e:
    print(e)
    
# Drop Artist Library table
try:
    query = "DROP TABLE artist_library"
    session.execute(query)
except Exception as e:
    print(e)
    
# Drop Album Library table
try:
    query = "DROP TABLE album_library"
    session.execute(query)
except Exception as e:
    print(e)

### Close the Session and Connection

In [34]:
session.shutdown()
cluster.shutdown()